In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import SubsetRandomSampler, DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
df_train = pd.read_csv('../input/Kannada-MNIST/train.csv')
df_test = pd.read_csv('../input/Kannada-MNIST/test.csv')
print('Train size: ', df_train.shape)
print('Test size: ', df_test.shape)
df_train.head()

In [ ]:
train_data = df_train.drop('label', axis=1).values
train_mean = train_data.mean()/255.
train_std = train_data.std()/255.
# train_std
print('Mean: ', train_mean)
print('Std: ', train_std)

In [ ]:
mask = np.random.rand(len(df_train)) < 0.8
df_val = df_train[~mask]
df_train = df_train[mask]
print('Train size: ', df_train.shape)
print('Val size: ', df_val.shape)
print('Test size: ', df_test.shape)
df_train.head()

In [ ]:
import matplotlib.pyplot as plt
ind = np.random.randint(0, df_train.shape[0]-1)
plt.imshow(df_train.iloc[ind].values[1:].reshape((28,28)), cmap='gray')
plt.title(str(df_train.iloc[ind][0]))

In [ ]:
class KannadaMNIST(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, n):
        data = self.df.iloc[n]
        image = data[1:].values.reshape((28,28)).astype(np.uint8)
        label = data[0]
        if self.transform:
            image = self.transform(image)
        return (image, label)

In [ ]:
transform = transforms.ToTensor()

In [ ]:
train_data = KannadaMNIST(df_train, transform = transform)

In [ ]:
batch_size = 20
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = num_workers)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images, images.shape, len(images), images[0].shape)
print()
print(labels, labels.shape, len(labels))

In [ ]:
fig =  plt.figure(figsize = (30,10))
for i in range(len(labels)):
  ax = fig.add_subplot(2,10,i+1,xticks=[], yticks = [])
  plt.imshow(np.squeeze(images[i]))
  ax.set_title(labels[i].item(), color = 'blue')

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,  input_size, hidden_dim, output_size):
    super(Discriminator, self).__init__()

    self.fc1 = nn.Linear(input_size, hidden_dim*4)
    self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim*2)
    self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim)
    self.fc4 = nn.Linear(hidden_dim, output_size)
    self.dropout = nn.Dropout(0.3)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.leaky_relu(self.fc1(x), 0.2) #negative_slope = 0.2
    x = self.dropout(x)
    x = F.leaky_relu(self.fc2(x), 0.2)
    x = self.dropout(x)
    x = F.leaky_relu(self.fc3(x), 0.2)
    x = self.dropout(x)
    out = self.fc4(x)

    return out

In [ ]:
class Generator(nn.Module):

  def __init__(self, input_size, hidden_dim, output_size):
    super(Generator, self).__init__()

    self.fc1 = nn.Linear(input_size, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, hidden_dim*2)
    self.fc3 = nn.Linear(hidden_dim*2, hidden_dim*4)
    self.fc4 = nn.Linear(hidden_dim*4, output_size)
    self.dropout = nn.Dropout(0.3)


  def forward(self, x):
    x = F.leaky_relu(self.fc1(x), 0.2)
    x = self.dropout(x)
    x = F.leaky_relu(self.fc2(x), 0.2)
    x = self.dropout(x)
    x = F.leaky_relu(self.fc3(x), 0.2)
    x = self.dropout(x)
    out = F.tanh(self.fc4(x))

    return out

In [ ]:
input_size = 784
d_output_size = 1
d_hidden_size = 32

In [ ]:
z_size = 100
g_output_size = 784
g_hidden_size = 32

In [ ]:
D = Discriminator(input_size, d_hidden_size, d_output_size)
G = Generator(z_size, g_hidden_size, g_output_size)

In [ ]:
print(D)
print(G)

In [ ]:
def real_loss(D_out, smooth = False):
  batch_size = D_out.size(0)
  if smooth:
    labels = torch.ones(batch_size) * 0.9
  
  else:
    labels = torch.ones(batch_size)

  criterion = nn.BCEWithLogitsLoss()
  loss = criterion(D_out.squeeze(1), labels)
  return loss

def fake_loss(D_out):
  batch_size = D_out.size(0)
  labels = torch.zeros(batch_size)
  criterion = nn.BCEWithLogitsLoss()

  loss = criterion(D_out.squeeze(1), labels)
  return loss

In [ ]:
import torch.optim as optim

lr = 0.002

d_optimizer = optim.Adam(D.parameters(), lr)
g_optimizer = optim.Adam(G.parameters(), lr)

In [ ]:
import pickle as pkl

num_epochs = 100

samples = []
losses = []

print_every = 400

sample_size = 16
fixed_z = np.random.uniform(-1, 1, size = (sample_size, z_size))
fixed_z = torch.from_numpy(fixed_z).float()

In [ ]:
D.train()
G.train()
for epoch in range(num_epochs):
  for batch_i, (real_images, _) in enumerate(train_loader):

    batch_size =  real_images.size(0)

    real_images = real_images * 2 - 1



    d_optimizer.zero_grad()


    # entrenar en imagenes reales

    D_real = D(real_images)
    d_real_loss = real_loss(D_real, smooth =  True)


    # entrenar en imagenes falsas
    z = np.random.uniform(-1, 1, size = (batch_size, z_size))
    z = torch.from_numpy(z).float()
    fake_images = G(z)

    #calcular la perdida del discriminador en iamgenes falsas
    D_fake = D(fake_images)
    d_fake_loss = fake_loss(D_fake)

    #realizar backprop
    d_loss = d_real_loss + d_fake_loss
    d_loss.backward()
    d_optimizer.step()



    #entrenamiento del generador

    g_optimizer.zero_grad()

    z = np.random.uniform(-1, 1, size=(batch_size, z_size))
    z = torch.from_numpy(z).float()
    fake_images = G(z)

    D_fake = D(fake_images)
    g_loss = real_loss(D_fake)

    g_loss.backward()
    g_optimizer.step()

    if batch_i % print_every == 0:
      print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format (epoch +1, num_epochs, d_loss.item(), g_loss.item()))

  losses.append((d_loss.item(), g_loss.item()))

  G.eval()
  samples_z = G(fixed_z)
  samples.append(samples_z)
  G.train()

with open('train_samples.pkl', 'wb') as f:
  pkl.dump(samples, f)

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label = 'Discriminator')
plt.plot(losses.T[1], label = 'Generator')
plt.title("Training Losses")
plt.legend()
plt.show()

In [ ]:
def view_samples(epoch, samples):
    fig, axes = plt.subplots(figsize=(7,7), nrows=4, ncols=4, sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        img = img.detach()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        im = ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        
        
with open('train_samples.pkl', 'rb') as f:
    samples = pkl.load(f)
    
view_samples(-1, samples)

In [ ]:

rows = 10 
cols = 6
fig, axes = plt.subplots(figsize=(7,12), nrows=rows, ncols=cols, sharex=True, sharey=True)

for sample, ax_row in zip(samples[::int(len(samples)/rows)], axes):
    for img, ax in zip(sample[::int(len(sample)/cols)], ax_row):
        img = img.detach()
        ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

In [ ]:
sample_size=16
rand_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
rand_z = torch.from_numpy(rand_z).float()

G.eval() 
rand_images = G(rand_z)
view_samples(0, [rand_images])